In [1]:
import networkx as nx

from regraph import Rule, plot_rule
from regraph.neo4j.graphs import Neo4jGraph
from regraph.neo4j.cypher_utils import *

In [2]:
# initialize the neo4j driver, wrappped into Neo4jGraph object
g = Neo4jGraph(uri="bolt://localhost:7687", user="neo4j", password="admin")
g._clear()

CREATE CONSTRAINT ON (n:node) ASSERT n.id IS UNIQUE
MATCH (n:node)
OPTIONAL MATCH (n)-[r]-()
DELETE n, r



In [3]:
g.add_nodes_from([
    ("n1", {"name": "Nils"}),
    ("n2", {"acmid": 220})
])
g.add_edge("n1", "n2", {"contact": True})


OPTIONAL MATCH (same_id_node:node) 
WHERE same_id_node.id = 'n1' 
FOREACH(new_count 
	IN CASE WHEN same_id_node IS NOT NULL
	THEN [coalesce(same_id_node.count, 0) + 1]
	ELSE [] END | 
		SET same_id_node.count = new_count) 
WITH same_id_node 
UNWIND
	CASE WHEN same_id_node IS NOT NULL
	THEN ['n1' + same_id_node.count]
	ELSE ['n1'] END AS new_id_n1 
		CREATE (n1:node { id : new_id_n1 }) 
SET n1.name=['Nils']
WITH new_id_n1, n1 OPTIONAL MATCH (same_id_node:node) 
WHERE same_id_node.id = 'n2' 
FOREACH(new_count 
	IN CASE WHEN same_id_node IS NOT NULL
	THEN [coalesce(same_id_node.count, 0) + 1]
	ELSE [] END | 
		SET same_id_node.count = new_count) 
WITH same_id_node 
UNWIND
	CASE WHEN same_id_node IS NOT NULL
	THEN ['n2' + same_id_node.count]
	ELSE ['n2'] END AS new_id_n2 
		CREATE (n2:node { id : new_id_n2 }) 
SET n2.acmid=[220]
WITH new_id_n2, n2 RETURN new_id_n2, n2

MATCH (n_n1:node { id : 'n1'}) , (n_n2:node { id : 'n2'})  MERGE (n_n1)-[new_edge:edge]->(n_n2)
SET new_edge.contact=[True

In [4]:
pattern = nx.DiGraph()
pattern.add_nodes_from(["a", "b", "c", "d"])
pattern.add_edges_from([("a", "c"), ("a", "d"), ("d", "c")])

In [5]:
rule = Rule.from_transform(pattern)

In [6]:
rule.inject_remove_edge("d", "c")
merge = rule.inject_merge_nodes(["c", "d"])
rule.inject_add_edge("b", merge)

In [7]:
instance = {
    "a": "n6",
    "b": "n10",
    "c": "n5",
    "d": "n9"
}

In [8]:
print(rule.to_cypher(instance)[0])

// Match nodes and edges of the instance 
MATCH (lhs_a:node { id : 'n6'}) , (lhs_b:node { id : 'n10'}) , (lhs_c:node { id : 'n5'}) , (lhs_d:node { id : 'n9'})  , (lhs_a)-[lhs_a_lhs_c:edge]->(lhs_c), (lhs_a)-[lhs_a_lhs_d:edge]->(lhs_d), (lhs_d)-[lhs_d_lhs_c:edge]->(lhs_c)

// Removing pattern matched edges 'd->c' of the lhs 
DELETE lhs_d_lhs_c

WITH lhs_a, lhs_b, lhs_a_lhs_d, lhs_c, lhs_d, lhs_a_lhs_c 
// Renaming vars to correspond to the vars of P
WITH lhs_a_lhs_d, lhs_a_lhs_c , lhs_a as p_a, lhs_b as p_b, lhs_c as p_c, lhs_d as p_d 

// Merging nodes '{'c', 'd'}' of the preserved part into 'c_d' 
// accumulate all the attrs of the nodes to be merged
WITH [] as new_props, p_b, p_d, p_c, lhs_a_lhs_d, p_a, lhs_a_lhs_c
WITH new_props + REDUCE(pairs = [], k in keys(p_c) | 
	pairs + REDUCE(inner_pairs = [], v in p_c[k] | 
		inner_pairs + {key: k, value: v})) as new_props, p_b, p_d, p_c, lhs_a_lhs_d, p_a, lhs_a_lhs_c
WITH new_props + REDUCE(pairs = [], k in keys(p_d) | 
	pairs + REDUCE(inne